<table align="left">
  <td>
    <a href="https://is.gd/2PddV3" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

 # Estudio Comparativo de Modelos de Clasificación para la Predicción de Churn

 - Regresion Logística - Softmax
 - Árboles de Decisión
 - Support Vector Machine  

